<a href="https://colab.research.google.com/github/DJCordhose/ai/blob/master/notebooks/process/export-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline and Export

* https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
* https://scikit-learn.org/0.21/modules/compose.html#column-transformer
* https://scikit-learn.org/stable/modules/model_persistence.html
* https://scikit-learn.org/0.21/modules/generated/sklearn.compose.ColumnTransformer.html

In [1]:
!rm -rf data && mkdir data && cd data && curl https://raw.githubusercontent.com/DJCordhose/deep-learning-crash-course-notebooks/master/data/insurance-customers-1500.csv -O && cd -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26783  100 26783    0     0   157k      0 --:--:-- --:--:-- --:--:--  157k
/content


In [2]:
!ls -l data

total 28
-rw-r--r-- 1 root root 26783 Jul 25 10:42 insurance-customers-1500.csv


In [0]:
import pandas as pd
df = pd.read_csv('data/insurance-customers-1500.csv', sep=';')

In [4]:
stats = df.describe()
stats

,speed,age,miles,group
count,1500.000000,1500.000000,1500.000000,1500.000000
mean,122.492667,44.980667,30.434000,0.998667
std,17.604333,17.130400,15.250815,0.816768
min,68.000000,16.000000,1.000000,0.000000
25%,108.000000,32.000000,18.000000,0.000000
50%,120.000000,42.000000,29.000000,1.000000
75%,137.000000,55.000000,42.000000,2.000000
max,166.000000,100.000000,84.000000,2.000000


In [0]:
y=df['group']

In [0]:
df.drop('group', axis='columns', inplace=True)

In [0]:
X = df.values

In [8]:
X

array([[ 98.,  44.,  25.],
       [118.,  54.,  24.],
       [111.,  26.,  34.],
       ...,
       [138.,  41.,  45.],
       [100.,  31.,  28.],
       [100.,  58.,  55.]])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1200, 3), (1200,), (300, 3), (300,))

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

In [0]:
column_trans = ColumnTransformer([], remainder=StandardScaler())

In [12]:
column_trans.fit_transform(X)

array([[-1.39175022, -0.05726625, -0.35642765],
       [-0.25528743,  0.52668598, -0.42201978],
       [-0.65304941, -1.10838025,  0.23390154],
       ...,
       [ 0.88117537, -0.23245192,  0.955415  ],
       [-1.27810395, -0.81640414, -0.15965125],
       [-1.27810395,  0.76026687,  1.61133633]])

In [0]:
pca = PCA()

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = clf = RandomForestClassifier(max_depth=9, min_samples_leaf=9, min_samples_split=3, n_estimators=7, n_jobs=-1)

In [0]:
clf = Pipeline([('standardise', column_trans), ('reduce_dim', PCA()), ('clf', rf_clf)])

In [17]:
%time clf.fit(X_train, y_train)

CPU times: user 27.5 ms, sys: 2.52 ms, total: 30.1 ms
Wall time: 122 ms


Pipeline(memory=None,
         steps=[('standardise',
                 ColumnTransformer(n_jobs=None,
                                   remainder=StandardScaler(copy=True,
                                                            with_mean=True,
                                                            with_std=True),
                                   sparse_threshold=0.3,
                                   transformer_weights=None, transformers=[],
                                   verbose=False)),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('clf',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=9,
                                        max_features='auto',
                                        max_lea

In [18]:
train_score = clf.score(X_train, y_train)
train_score

0.8291666666666667

In [19]:
test_score = clf.score(X_test, y_test)
test_score

0.7833333333333333

In [20]:
from sklearn.model_selection import cross_val_score
cross_val_scores = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=10)
cross_val_scores

array([0.76033058, 0.775     , 0.76666667, 0.775     , 0.65833333,
       0.76666667, 0.83333333, 0.80833333, 0.79166667, 0.80672269])

In [21]:
# mean score and the 95% confidence interval (2 standard deviations assuming normal distribution)
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_scores.mean(), cross_val_scores.std() * 2))

Accuracy: 0.77 (+/- 0.09)


In [22]:
scores = {
    'cross_val_scores': cross_val_scores,
    'train_score': train_score,
    'test_score': test_score
}
scores

{'cross_val_scores': array([0.76033058, 0.775     , 0.76666667, 0.775     , 0.65833333,
        0.76666667, 0.83333333, 0.80833333, 0.79166667, 0.80672269]),
 'test_score': 0.7833333333333333,
 'train_score': 0.8291666666666667}

In [0]:
!rm -rf models && mkdir models

In [0]:
import pickle

pickle.dump(clf, open('models/model.pickle', 'wb'))

In [0]:
!rm -rf stats && mkdir stats

In [0]:
# has the distribution of the data changed substantially?
pickle.dump(stats, open('stats/describe.pickle', 'wb'))

In [0]:
# has the score descread since last time or is suspiciously different?
pickle.dump(scores, open('stats/scores.pickle', 'wb'))

In [28]:
import numpy, sklearn, pandas

versions = {
    'numpy': numpy.__version__,
    'sklearn': sklearn.__version__, 
    'pandas': pandas.__version__,
    'model': 2,
    'model_name': 'pipeline_pca_std_rf'
}
versions

{'model': 2,
 'model_name': 'pipeline_pca_std_rf',
 'numpy': '1.16.4',
 'pandas': '0.24.2',
 'sklearn': '0.21.2'}

In [0]:
# we need to have exactly the same versions for serving
pickle.dump(versions, open('stats/versions.pickle', 'wb'))

In [30]:
!ls -l stats

total 12
-rw-r--r-- 1 root root 1145 Jul 25 10:43 describe.pickle
-rw-r--r-- 1 root root  372 Jul 25 10:43 scores.pickle
-rw-r--r-- 1 root root  143 Jul 25 10:43 versions.pickle


In [31]:
!ls -l

total 136
drwxr-xr-x 2 root root  4096 Jul 25 10:42 data
-rw-r--r-- 1 root root 26688 Jul 25 10:29 export.ipynb
-rw-r--r-- 1 root root    15 Jul 25 10:35 export-pipeline.ipynb
drwxr-xr-x 2 root root  4096 Jul 25 10:42 models
-rw-r--r-- 1 root root 89706 Jul 25 10:35 prod.tgz
drwxr-xr-x 1 root root  4096 Jul 19 16:14 sample_data
drwxr-xr-x 2 root root  4096 Jul 25 10:43 stats


In [32]:
# we also need the code that created this to have it reproduceable
!cd models
!curl https://raw.githubusercontent.com/DJCordhose/ml-workshop/master/notebooks/process/export-pipeline.ipynb -O
!cd -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0     87      0 --:--:-- --:--:-- --:--:--    87
/


In [33]:
!ls -l models

total 68
-rw-r--r-- 1 root root 66179 Jul 25 10:42 model.pickle


In [0]:
# have everything we need to reproduce and compare to other versions here
# better to version this 
# often versioning data is a challenge simply because of its size
!tar -czf prod.tgz models stats data

In [35]:
!ls -l

total 76
drwxr-xr-x 2 root root  4096 Jul 25 10:42 data
-rw-r--r-- 1 root root 26688 Jul 25 10:29 export.ipynb
-rw-r--r-- 1 root root    15 Jul 25 10:43 export-pipeline.ipynb
drwxr-xr-x 2 root root  4096 Jul 25 10:42 models
-rw-r--r-- 1 root root 26565 Jul 25 10:43 prod.tgz
drwxr-xr-x 1 root root  4096 Jul 19 16:14 sample_data
drwxr-xr-x 2 root root  4096 Jul 25 10:43 stats


In [36]:
!date

Thu Jul 25 10:43:15 UTC 2019


# Exercise: Chain together different / additional steps in a pipeline

* can you add a normaizer?
* PCA to less dimensions?
* different 
